# Sentinel 2 CH4 Multi Band Multi Pass Mapper

## Overview
This code displays methane clouds over a ladfill with a 20m2 resolution. It runs the analysis following the multi-band-multi-pass retrieval method proposed by Varon et al. (2021), albeit without their radiative transfer model, which was unfortunately not provided in their research paper. The authors of the paper have been contacted, and this code can be added in the future. 

The code uses two seperate datasets. One for a day with an active emission, and another for a no emission day. Here a multi-band-single-pass calculation for both active emission and no emission dates is performed, resulting in two datasets. 
The multi-band-single-pass equation is as follows: 

<div align="center"><b>ΔRMBSP= cR12 − R11 / R11</b></div>


Where:
- c is calculated using a least-squares fitting algorithm of R12 against R11. 
- R12 is the Sentinel-2 SWIR-2 band (“B12” in the code).
- R11 is the Sentinel-2 SWIR-1 band (“B11” in the code). 

To calculate the correction factor ‘c’ a Python code developed by staff at Amazon Web Services for a similar CH4 model was used (AWS, 2023). The process is as follows: 
- ‘compute_correction_factor’ performs a linear least square regression using the SWIR-2 and SWIR-1 datasets from both active and no emission .Gtiff files. 
- ‘compute_corrected_fractional_reflectance_change’ applies the computed correction factor and applies it to the active emission scene (R12) to give cR12.

Once the multi-band-single-pass has been calculated for both active emission and no emission dates, the no emission scene is subtracted from the active emission scene: 

<div align="center"><b>ΔΩMBMP = ΔΩMBSP − ΔΩ′MBSP</b></div>

Where:
- ΔΩMBSP is the multiband single pass for the active emission scene.
- ΔΩ′MBSP is the multiband single pass for the no emission scene. 

The dates for the 'active_emission_data' and 'no_emission_data' can be determined by using the Sentinel 5P Time Series, or Sentinel 5P CH4 Map tools. If you run into any difficulty, please refer to section 4.3 in the How to guide. 

The section below imports the packages needed to run the script.

In [ ]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
import openeo
import rasterio
import numpy as np
import requests
from sklearn.linear_model import LinearRegression

## Connect to OpenEO

The code below establishes a connection with the Copernicus openEO platform which provides a wide variety of earth observation datasets

- If this does not read as 'Authorised successfully' or 'Authenticated using refresh token', then please ensure that you have completed the setup steps as outlined in section 2.6 of the user guide. 

- If you have followed the steps in section 2.6 correctly and the problem persists, please look at https://dataspace.copernicus.eu/news for any information about service interruptions. 

- If there is no news of service problems you can raise a ticket here: https://helpcenter.dataspace.copernicus.eu/hc/en-gb/requests/new

In [ ]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

## View list of landfills. 

Each landfill has been asigned an id number. This number can be found in the list produced by the following code. To look at a chosen landfill, you will need to input that number at various stages in the code. Run the code below to the the list. The west, south, east and north columns are latitude and longitude cordinates to establish a box around each landfill for analysis.  

There are two bounding extents files for the tool, small and large. 
- use '...\PreZero_Landfill_Bounding_small.csv' for small.
- use '...\PreZero_Landfill_Bounding_small.csv' for large.

Please place a # next to the one you DO NOT wish to use.

In [ ]:
landfill_csv = pd.read_csv(r'C:\GIS_Course\EGM722\Project\Data\PreZero_Landfill_Bounding_small.csv')
# landfill_csv = pd.read_csv(r'C:\GIS_Course\EGM722\Project\Data\PreZero_Landfill_Bounding_small.csv')
landfill_csv

# Available dates for the analysis. 

Unlike Sentinel 5P, Sentinel 2 only provides data aproximately once every 3 days. Because of this, not every date you can enter into this tool is valid. The code below will tell you what dates are available to use for the landfill of your choice. 

The two parameters you need to modify before running the code are: 
- landfill_id = 23 (change this to your chosen landfill) 
- temporal_extent = ["2023-01-31", "2023-03-12"] (change this to your chosen date range using "YYYY-MM-DD" format)

Once you have done this run the code and the available dates should appear below in a matter of a few seconds. 

In [ ]:
def get_spatial_extent(landfill_id):
    landfill = landfill_csv[landfill_csv['id'] == landfill_id].iloc[0]
    return {
        "west": landfill['west'],
        "south": landfill['south'],
        "east": landfill['east'],
        "north": landfill['north']
    }

def fetch_available_dates(landfill_id, temporal_extent):
    spatial_extent = get_spatial_extent(landfill_id)
    catalog_url = f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?box={spatial_extent['west']}%2C{spatial_extent['south']}%2C{spatial_extent['east']}%2C{spatial_extent['north']}&sortParam=startDate&sortOrder=ascending&page=1&maxRecords=1000&status=ONLINE&dataset=ESA-DATASET&productType=L2A&startDate={temporal_extent[0]}T00%3A00%3A00Z&completionDate={temporal_extent[1]}T00%3A00%3A00Z&cloudCover=%5B0%2C{cloud_cover}%5D"
    response = requests.get(catalog_url)
    response.raise_for_status()
    catalog = response.json()
    dates = [date.split('T')[0] for date in map(lambda x: x['properties']['startDate'], catalog['features'])]
    return dates

# Please enter your perameters here.
landfill_id = 23  # Specify the landfill ID.
temporal_extent = ["2023-01-01", "2023-03-12"]  # Specify the the date range you want to check for available data.
cloud_cover = 15

available_dates = fetch_available_dates(landfill_id, temporal_extent)
print("Available dates:", available_dates)

## Choosing the Active Emission Date

As mentioned in the overview, an active emission date must be chosen from one of the available datasets. 

Like before, the two parameters you need to modify before running the code are:

- landfill_id = 23 (change this to your chosen landfill)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.) 

Please note that the temporal extent dates <u>MUST BE IDENTICAL</u> because we are only choosing a single date.

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.


In [ ]:
def active_emission(landfill_id, temporal_extent):
    landfill = landfill_csv[landfill_csv['id'] == landfill_id].iloc[0]

    active_emission = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": landfill['west'],
            "south": landfill['south'],
            "east": landfill['east'],
            "north": landfill['north']
        },
        bands=["B11", "B12"],
    )
    active_emission.download("Sentinel-2_active_emissionMBMP.GTiff")

# Enter perameters for the active emission day
landfill_id = 23  # Specify the landfill ID
temporal_extent = ["2023-02-25", "2023-02-25"]

active_emission(landfill_id, temporal_extent)

## Choosing the No Emission Date

Next we choose the no emission date using the same process. 

The two parameters you need to modify before running the code are:

- landfill_id = 23 (change this to your chosen landfill)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.) 

The temporal extent dates <u>MUST BE IDENTICAL</u>

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.


In [ ]:
def no_emission(landfill_id, temporal_extent):
    landfill = landfill_csv[landfill_csv['id'] == landfill_id].iloc[0]

    no_emission_collection = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": landfill['west'],
            "south": landfill['south'],
            "east": landfill['east'],
            "north": landfill['north']
        },
        bands=["B11", "B12"],
    )
    no_emission_collection.download("Sentinel-2_no_emissionMBMP.GTiff")

# Enter parameters for the no emission day
landfill_id = 23  # Specify the landfill ID
temporal_extent = ["2023-02-10", "2023-02-10"]

no_emission(landfill_id, temporal_extent)

## Choosing A Background Satelite Image

This section helps with locating the source of the emission at the landfill by displaying a true colour satelite image of the landfill that the data will be superimposed over. I recommend choosing the same date as your active emission. 

Once again, the two parameters you need to modify before running the code are:

- landfill_id = 23 (change this to your chosen landfill)
- temporal_extent = ["2023-02-25", "2023-02-25"] (change this to your chosen date range using "YYYY-MM-DD" format.)

The temporal extent dates <u>MUST BE IDENTICAL</u>

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.

In [ ]:
def truecolour_image(landfill_id, temporal_extent):
    landfill = landfill_csv[landfill_csv['id'] == landfill_id].iloc[0]

    truecolour_image_collection = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=temporal_extent,
        spatial_extent={
            "west": landfill['west'],
            "south": landfill['south'],
            "east": landfill['east'],
            "north": landfill['north']
        },
        bands=["B02", "B03", "B04"],
    )
    truecolour_image_collection.download("Sentinel-2_truecolour.GTiff")

# Enter parameters for the no emission day
landfill_id = 23  # Specify the landfill ID
temporal_extent = ["2023-02-25", "2023-02-25"]

truecolour_image(landfill_id, temporal_extent)

In [ ]:
Active_Multiband = "Sentinel-2_active_emissionMBMP.GTiff"
Active_img = rasterio.open(Active_Multiband)
Active_B11 = Active_img.read(1)
Active_B12 = Active_img.read(2)

No_Multiband = "Sentinel-2_no_emissionMBMP.GTiff"
No_img = rasterio.open(No_Multiband)
No_B11 = No_img.read(1)
No_B12 = No_img.read(2)

## Running the Analysis
Here, no modification by you is needed. The code below runs the analysis.  Provided all the variables above have been run correctly, this next section should take moments to complete. 

In [ ]:
# Multi band Single Pass Active
def active_compute_correction_factor(Active_B12, Active_B11):
    # get flattened arrays for regression
    yB12 = Active_B12.flatten()
    xB11 = Active_B11.flatten()

    # replace NaN values with 0
    yB12 = np.nan_to_num(yB12)
    xB11 = np.nan_to_num(xB11)

    assert yB12.shape == xB11.shape, "Shapes of two provided arrays are incompatible!"

    # fit linear model using least squares regression
    xB11 = xB11[:, np.newaxis]
    c, _, _, _ = np.linalg.lstsq(xB11, yB12, rcond=None)

    return c[0]

def active_compute_corrected_fractional_reflectance_change(Active_B12, Active_B11):
    # get correction factor
    c = active_compute_correction_factor(Active_B12, Active_B11)

    # get corrected fractional reflectance change
    active_frac_change = ((c * Active_B12 - Active_B11) / Active_B11)

    return active_frac_change

# Open raster files
with rasterio.open(r"C:\GIS_Course\EGM722\Project\Sentinel-2_active_emissionMBMP.GTiff") as B12_active_emission_file:
    B12_active_emission_data = B12_active_emission_file.read(2)
with rasterio.open(r"C:\GIS_Course\EGM722\Project\Sentinel-2_active_emissionMBMP.GTiff") as B11_active_emission_file:
    B11_active_emission_data = B11_active_emission_file.read(1)

# Compute corrected fractional reflectance change
active_frac_change = active_compute_corrected_fractional_reflectance_change(B12_active_emission_data, B11_active_emission_data)

# Multi band Single Pass No
def no_compute_correction_factor(B12_no_emission_data, B11_no_emission_data):
    # get flattened arrays for regression
    yB12 = No_B12.flatten()
    xB11 = No_B11.flatten()

    # replace NaN values with 0
    yB12 = np.nan_to_num(yB12)
    xB11 = np.nan_to_num(xB11)

    assert yB12.shape == xB11.shape, "Shapes of two provided arrays are incompatible!"

    # fit linear model using least squares regression
    xB11 = xB11[:, np.newaxis]
    c, _, _, _ = np.linalg.lstsq(xB11, yB12, rcond=None)

    return c[0]

def no_compute_corrected_fractional_reflectance_change(No_B12, No_B11):
    # get correction factor
    c = no_compute_correction_factor(No_B12, No_B11)

    # get corrected fractional reflectance change
    no_frac_change = ((c * No_B12 - No_B11) / No_B11)

    return no_frac_change

# Open raster files
with rasterio.open(r"C:\GIS_Course\EGM722\Project\Sentinel-2_active_emissionMBMP.GTiff") as B12_no_emission_file:
    B12_no_emission_data = B12_no_emission_file.read(2)
with rasterio.open(r"C:\GIS_Course\EGM722\Project\Sentinel-2_no_emissionMBMP.GTiff") as B11_no_emission_file:
    B11_no_emission_data = B11_no_emission_file.read(1)

# Compute corrected fractional reflectance change
no_frac_change = no_compute_corrected_fractional_reflectance_change(B12_no_emission_data, B11_no_emission_data)


frac_change = active_frac_change - no_frac_change

## Viewing the data. 

This section of code can be run to produce the map. There is one piece of code that can be altered if needed. 

- brightness_factor = 0.05 (occasionally the true colour satelite image can be too bright or too dark. You can change this number to fix it)

In [ ]:
# Open the and load the truecolour satellite image file and define which band is which.
fp = 'Sentinel-2_truecolour.GTiff'
img = rasterio.open(fp)
blue = img.read(1)
green = img.read(2)
red = img.read(3)

# Change this number up or down if the satellite background image is too dark or bright. 
brightness_factor = 0.05 
blue = np.clip(blue * brightness_factor, 0, 255)
green = np.clip(green * brightness_factor, 0, 255)
red = np.clip(red * brightness_factor, 0, 255)

# Stack the blue, green and red bands to make a colour image. 
rgb = np.dstack((red, green, blue))
rgb = rgb / rgb.max()

# Create a new figure
plt.figure(figsize=(10, 8))

# Display the RGB image
plt.imshow(rgb)

# Loading the methane data.
# Create a masked array where values below 0.01 are masked
masked_frac_change = np.ma.masked_greater(frac_change, -0.1)

# Define the range of your colormap
vmin = -5
vmax = 0

# Display the masked_frac_change data on top of the RGB image
plt.imshow(masked_frac_change, cmap='plasma_r', alpha=1, vmin=vmin, vmax=vmax)

# Add a colorbar and labels
cbar = plt.colorbar(ticks=[vmin, vmax], label='CH4 Levels', shrink=0.7)
cbar.ax.set_yticklabels(['Higher', 'Lower'])  # set ticks labels
plt.title('CH4 Levels')

# Dataset resolution in metres
resolution = 20  

# Get the dimensions of the image
height, width, _ = rgb.shape

# Create arrays representing the x and y coordinates in meters
x = np.arange(0, width * resolution, 5000)
y = np.arange(0, height * resolution, 5000)

# Set the x and y ticks and labels
plt.xticks(x / resolution, x)
plt.yticks(y / resolution, y)
plt.xlabel('X (meters)')
plt.ylabel('Y (meters)')

# Add grid lines
plt.grid(color='gray', linestyle='-', linewidth=0.5)

# Please enter the emission date where it says '[emission date here]' if you want to download more than one map.
plt.savefig(f'S2HRM[emission_date_here].jpg', format='jpg', bbox_inches='tight')

# Show the plot
plt.show()


## Reference:

Varon, D. J., Jervis, D., McKeever, J., Spence, I., Gains, D., & Jacob, D. J. (2021). High-frequency monitoring of anomalous methane point sources with multispectral Sentinel-2 satellite observations. Atmospheric Measurement Techniques, 14, 2771–2785. 